In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import seaborn as sns

%matplotlib inline

In [3]:
# Import CSV file of ETFs

etf_df = pd.read_csv('ETF prices.csv', parse_dates=True, infer_datetime_format=True)

In [4]:
# Drop unneeded columns from DataFrame

etf_df.drop(columns=['open', 'high', 'low', 'close', 'volume'], inplace=True)

In [5]:
# Create dataframe table with data starting from 2016

etf_2016 = etf_df.loc[etf_df['price_date'] >= "2016"]

In [7]:
# Create pivot table that displays the prices of each ETF by date

etf_pivot = etf_2016.pivot_table(index = ["price_date"], columns = "fund_symbol", values = "adj_close")

In [10]:
# Remove ETFs that don't have a full data sample over entire 5-year period

etf_pivot_df = etf_pivot.dropna(axis='columns')

In [12]:
#Find the daily returns of each ETF

etf_returns = etf_pivot_df.pct_change().dropna()

In [72]:
# Import CSV file of 10-year Treasury rates data

tnx_df = pd.read_csv("^TNX.csv", parse_dates=True, infer_datetime_format=True)

In [76]:
# Set df index

tnx_index = tnx_df.set_index("Date")

In [81]:
# Drop irrelevant columns

tnx_index.drop(columns=['Open', "High", "Low", "Close", "Volume"], inplace=True)

In [82]:
# Drop Null Values

tnx_df.dropna(inplace=True)

In [83]:
# Concatenate TNX df with ETF df

tnx_etf = pd.concat([tnx_index, etf_pivot_df], join = "inner", axis = 1)

In [86]:
# Rename TNX column

tnx_etf.rename(columns={"Adj Close" : "TNX"}, inplace=True)

In [90]:
# Find daily returns of Dataframe and drop null values

tnx_etf_returns = tnx_etf.pct_change().dropna()

In [98]:
# Find sharpe ratio of each ETF

sharpe_ratio = ((tnx_etf_returns.mean()-tnx_etf_returns['TNX'].mean()) * 252)/(tnx_etf_returns.std() * np.sqrt(252))

In [108]:
# Import ETF CSV

etf_desc = pd.read_csv("ETFs.csv")

In [110]:
# Match etfs with etfs that contain 5 years of data

etf_matched = etf_desc[etf_desc.fund_symbol.isin(tnx_etf_returns.columns)]

In [164]:
# Drop leveraged ETFs

dropped = etf_matched[etf_matched["fund_category"].str.contains("Leveraged")==False]

In [169]:
# Drop Ultrashort ETFs

ultra_etf = dropped[dropped['fund_category'].str.contains("Ultrashort")==False]

In [189]:
# Drop Inverse ETFs and Display total ETF selection

final_etf = ultra_etf[ultra_etf['fund_category'].str.contains("Inverse")==False]

In [195]:
# Factor out Bond ETFs from Equity ETFs

bond_etfs = final_etf[final_etf.fund_category.str.contains('Bond', na=False)]

In [203]:
# Factor out Equity ETFs from Bond ETFs

equity_etfs = final_etf[final_etf["fund_category"].str.contains('Bond')==False]

In [400]:
# Create column that returns a bool for real estate

real_estate = equity_etfs["fund_category"] == "Real Estate"

In [514]:
# Add real estate column to df

equity_etfs["Real Estate"] = real_estate

/var/folders/xf/5gg0rlzs2xlbl_dwfhdrq43h0000gn/T/ipykernel_11455/2214581544.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  equity_etfs["Real Estate"] = real_estate


In [251]:
# Factor out Foreign from Domestic Equities

foreign_equity = equity_etfs[(equity_etfs.fund_category.str.contains('Foreign') | (equity_etfs.fund_category.str.contains('Pacific/Asia ex-Japan Stk'))| (equity_etfs.fund_category.str.contains('Emerging'))
            | (equity_etfs.fund_category.str.contains('China')) | (equity_etfs.fund_category.str.contains('Latin')) | (equity_etfs.fund_category.str.contains('Europe')) | (equity_etfs.fund_category.str.contains('Japan')) |
                             (equity_etfs.fund_category.str.contains('India'))]

In [254]:
# Factor out Real Estate ETFs

real_estate_etfs = equity_etfs[equity_etfs.fund_category.str.contains("Real Estate")]

In [431]:
# Filter equities by betas

equity_beta = equity_etfs[['fund_symbol', 'fund_beta_5years', 'fund_sharpe_ratio_5years']]

In [432]:
# Filter out highly negative betas

equity_etf_beta = equity_beta[equity_beta.fund_beta_5years > -4]

In [433]:
equity_beta

,fund_symbol,fund_beta_5years,fund_sharpe_ratio_5years
1,AADR,1.11,0.62
2,AAXJ,0.94,0.66
10,ACWX,1.00,0.70
13,ADRE,1.10,0.57
17,AFK,1.19,0.28
...,...,...,...
2305,EPP,1.07,0.48
2306,ERUS,1.19,0.70
2307,EWA,1.16,0.48
2308,EWC,1.10,0.61


In [434]:
# Create df of lower beta ETFs

low_beta = equity_beta[(equity_beta.fund_beta_5years > 0) & (equity_beta.fund_beta_5years < 0.5)]

In [435]:
# Create df of low to medium beta ETFs

low_medium_beta = equity_beta[(equity_beta.fund_beta_5years > 0.51) & (equity_beta.fund_beta_5years < 1.00)]

In [436]:
# Create df of medium level beta ETFs

medium_beta = equity_beta[(equity_beta.fund_beta_5years > 1.001) & (equity_beta.fund_beta_5years < 1.5)]

In [437]:
# Create df of medium to high ETFs

high_medium_beta = equity_beta[(equity_beta.fund_beta_5years > 1.25) & (equity_beta.fund_beta_5years < 1.75)]

In [438]:
# Create df of high beta ETFs

high_beta = equity_beta[(equity_beta.fund_beta_5years > 1.76) & (equity_beta.fund_beta_5years < 5)]

In [333]:
# Risk Tolerances (by basket)

1 - 40%, 30%, 20%, 5%, 5%
2 - 20%, 40%, 20%, 10%, 10%
3 - 10%, 20%, 40%, 20%, 10%
4 - 10%, 10%, 20%, 40%, 20%
5 - 5%, 5%, 20%, 30%, 40%

In [373]:
# Find bond investment grade qualities

bond_quality = bond_etfs[['fund_symbol', 'fund_bonds_aaa', 'fund_bonds_aa', 'fund_bonds_a', 'fund_bonds_bbb', 
          'fund_bonds_bb', 'fund_bonds_b', 'fund_bonds_below_b']]

In [378]:
# Drop NaN's

bond_quality = bond_quality.dropna()

In [363]:
# Find what percent of bond ETFs are investment grade

investment_percent = bond_quality['fund_bonds_aaa'] + bond_quality['fund_bonds_aa'] + bond_quality['fund_bonds_a']

In [387]:
# Find what percent of bond ETFs are mid-investment grade

mid_investment = bond_quality['fund_bonds_bbb'] 

In [388]:
# Find what percent of bond ETFs are non-investment grade

noninvest_percent = bond_quality['fund_bonds_bb'] + bond_quality['fund_bonds_b']

In [389]:
# Add column to df

bond_quality["Investment Grade Percent"] = investment_percent

/var/folders/xf/5gg0rlzs2xlbl_dwfhdrq43h0000gn/T/ipykernel_11455/3901423594.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bond_quality["Investment Grade Percent"] = investment_percent


In [392]:
bond_quality['Mid Investment Grade'] = mid_investment

/var/folders/xf/5gg0rlzs2xlbl_dwfhdrq43h0000gn/T/ipykernel_11455/2863926551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bond_quality['Mid Investment Grade'] = mid_investment


In [390]:
# Add column to df

bond_quality["Non-Invest Grade Percent"] = noninvest_percent

/var/folders/xf/5gg0rlzs2xlbl_dwfhdrq43h0000gn/T/ipykernel_11455/1515454792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bond_quality["Non-Invest Grade Percent"] = noninvest_percent


In [411]:
# Create a basket of low-risk bonds

low_risk_bonds = bond_quality[bond_quality["Investment Grade Percent"] > .75]

In [415]:
# Create a basket of high-risk bonds

high_risk_bonds = bond_quality[bond_quality["Non-Invest Grade Percent"] > .75]

In [424]:
# Create a basket of medium-risk bonds

mid_risk_bonds = bond_quality[(bond_quality["Investment Grade Percent"] < 0.75) & 
                              (bond_quality["Investment Grade Percent"] > 0.45)]

In [ ]:
# Bond allocation - low, mid, high

low tolerance = 85%, 10%, 5%
medium tolerance = 75%, 15%, 10%
high tolerance = 65%, 20%, 15%


In [ ]:
# Baskets
low_beta
low_medium_beta
medium_beta
high_medium_beta
high_beta

real_estate_etfs
foreign_equity

low_risk_bonds
mid_risk_bonds
high_risk_bonds

In [474]:
# Create etf basket for lowest risk

low_beta_equity_basket = low_beta.sort_values(by='fund_sharpe_ratio_5years', ascending=False).head(10)

In [475]:
# Create etf basket for low risk

low_medium_beta_equity_basket = low_medium_beta.sort_values(by='fund_sharpe_ratio_5years', ascending=False).head(10)

In [476]:
# Create etf basket for medium risk

medium_beta_equity_basket = medium_beta.sort_values(by='fund_sharpe_ratio_5years', ascending=False).head(11).dropna()

In [477]:
# Create etf basket for high risk

high_medium_beta_equity_basket = high_medium_beta.sort_values(by='fund_sharpe_ratio_5years', ascending=False).head(11).dropna()

In [478]:
# Create etf basket for highest risk

high_beta_equity_basket = high_beta.sort_values(by='fund_sharpe_ratio_5years', ascending=False).head(10)

In [496]:
# Create real estate basket

real_estate_basket = real_estate_etfs[['fund_symbol' , "fund_sharpe_ratio_5years"]].sort_values(by='fund_sharpe_ratio_5years', 
                                                                                                ascending=False).head(7)

In [497]:
# Create foreign equity etf basket

foreign_equity_basket = foreign_equity[['fund_symbol' , "fund_sharpe_ratio_5years"]].sort_values(by='fund_sharpe_ratio_5years', 
                                                                                                 ascending=False).head(15)